# working model for tensorfusion

In [1]:
import pickle
import torch
from torchmetrics import F1Score
import pandas as pd
import numpy as np
from glob import glob
from collections import Counter
import torch.nn.functional as F
from tqdm import tqdm
import os
from datasets import Dataset
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import random

print(torch.__version__)

/home/arplab/project/lou/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


## wav, text데이터 불러오기

In [2]:
# wav and text data load
dataset_file_lst = ['../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl',
                    '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl',
                    '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl',
                    '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl']
dataset_file_lst = sorted(dataset_file_lst)
dataset_file_lst


['../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl',
 '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl',
 '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl',
 '../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl']

In [3]:

var_names = ['kemdy19_annot', 'kemdy19_emb', 'kemdy20_annot', 'kemdy20_emb']
for file, var_name in zip(dataset_file_lst, var_names):
    print(file, var_name)
    with open(file, 'rb') as f:
        globals()[var_name] = pickle.load(f)

../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl kemdy19_annot
../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY19_embedding_for_dataset.pkl kemdy19_emb
../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_annotation_nonmissing.pkl kemdy20_annot
../../../paradeigma/multi_modal/model/data/paradeigma_KEMDY20_embedding_for_dataset.pkl kemdy20_emb


In [4]:
#  paradeigma가 섞어놓은 label들..되돌리기
# {'neutral': 0, 'happy': 0, 'surprise':0, 'disgust': 0, 'angry': 0, 'sad':0, 'fear': 0}
# {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

def change_order(lst, new_order = [4, 3, 6, 1, 0, 5, 2]):
    return [lst[i] for i in new_order]

kemdy20_annot['emotion_vector'] = kemdy20_annot['emotion_vector'].apply(change_order)
kemdy19_annot['emotion_vector'] = kemdy19_annot['emotion_vector'].apply(change_order)

In [5]:
kemdy19_annot.head(3)

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector,EDA,TEMP,EDA length,TEMP length,Scaled EDA,Scaled TEMP
0,Sess01_script01_M001,surprise,1.7,4.0,"[0, 0, 1, 0, 0, 0, 9]","[4, 5, 1, 0, 0]","[1, 0, 0, 6, 3]","[4.408951, 4.403825, 4.410233, 4.421767, 4.429...","[34.66, 34.66, 34.66, 34.66, 34.66, 34.68, 34....",33,33,"[-0.926455937246618, -0.9285481420682529, -0.9...","[1.6900874192514124, 1.6900874192514124, 1.690..."
1,Sess01_script01_F001,fear,1.5,3.6,"[0, 0, 7, 0, 1, 0, 2]","[5, 5, 0, 0, 0]","[1, 0, 2, 6, 1]","[0.145914, 0.140794, 0.144634, 0.145914, 0.140...","[30.69, 30.69, 30.69, 30.69, 30.71, 30.71, 30....",17,17,"[-1.407155706494133, -1.4312942323175069, -1.4...","[-1.1135528407658513, -1.1135528407658513, -1...."
2,Sess01_script01_M002,angry,1.3,4.3,"[10, 0, 0, 0, 0, 0, 0]","[7, 3, 0, 0, 0]","[1, 0, 0, 3, 6]","[4.478828, 4.396809, 4.334012, 4.322478, 4.346...","[34.61, 34.61, 34.61, 34.61, 34.61, 34.61, 34....",27,27,"[-0.8979352590723317, -0.9314117606848334, -0....","[1.6164861115016742, 1.6164861115016742, 1.616..."


<2020년>
'angry;disqust',  'angry;disqust;fear;neutral;sad',  'angry;disqust;neutral', 'angry;happy;neutral', 'angry;neutral', 'disqust;happy;neutral', 'disqust;neutral', 'disqust;neutral;sad', 'fear;happy', 'fear;happy;neutral', 'fear;neutral', 'happy;neutral', 'happy;neutral;surprise', 'happy;sad', 'happy;surprise', 'neutral;sad', 'neutral;surprise'

<2019년>
'angry;disgust;fear;neutral;surprise', 'angry;fear', 'angry;fear;neutral', 'angry;fear;surprise', 'angry;happy', 'angry;neutral;surprise', 'angry;sad', 'angry;surprise', 'disgust;fear', 'disgust;happy','disgust;neutral;surprise', 'disgust;sad', 'disgust;surprise', 'fear;neutral;surprise', 'fear;sad', 'fear;surprise', 'happy;neutral;sad', 'neutral;sad;surprise', 'sad;surprise'}


In [6]:
encode_dict = {'angry':0, 'disgust':1, 'fear':2,'happy':3,'neutral':4, 'sad':5, 'surprise':6,  
               'neutral;surprise': 450, 'neutral;sad': 460, 'happy;neutral': 340, 
               'angry;neutral': 40, 'disgust;neutral': 140, 'fear;neutral': 240, 
               'happy;surprise': 350, 'angry;happy;neutral': 7340, 'angry;disgust': 10, 
               'happy;neutral;surprise': 3450, 'fear;happy': 230,'fear;happy;neutral': 2340,
               'angry;disgust;neutral': 7140, 'disgust;neutral;sad': 1460, 
               'happy;sad': 360, 'disgust;happy;neutral': 3410, 'angry;fear': 20, 'angry;fear;neutral':7240,
               'angry;fear;surprise': 7250, 'angry;happy': 730, 'angry;neutral;surprise':7450, 
               'angry;sad': 60, 'angry;surprise': 50, 'disgust;fear':120, 'disgust;happy': 130,
               'disgust;neutral;surprise':1450, 'disgust;sad': 160, 'disgust;surprise':150, 
               'fear;neutral;surprise':2450, 'fear;sad': 260, 'fear;surprise':250, 'happy;neutral;sad':3460,
               'neutral;sad;surprise':4560, 'sad;surprise': 560,
               'angry;disgust;fear;neutral;sad': 10000,'angry;disgust;fear;neutral;surprise':20000}

In [7]:
encode_dict = {k: v for k, v in sorted(encode_dict.items(), key=lambda item: item[1])}
print(encode_dict)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6, 'angry;disgust': 10, 'angry;fear': 20, 'angry;neutral': 40, 'angry;surprise': 50, 'angry;sad': 60, 'disgust;fear': 120, 'disgust;happy': 130, 'disgust;neutral': 140, 'disgust;surprise': 150, 'disgust;sad': 160, 'fear;happy': 230, 'fear;neutral': 240, 'fear;surprise': 250, 'fear;sad': 260, 'happy;neutral': 340, 'happy;surprise': 350, 'happy;sad': 360, 'neutral;surprise': 450, 'neutral;sad': 460, 'sad;surprise': 560, 'angry;happy': 730, 'disgust;neutral;surprise': 1450, 'disgust;neutral;sad': 1460, 'fear;happy;neutral': 2340, 'fear;neutral;surprise': 2450, 'disgust;happy;neutral': 3410, 'happy;neutral;surprise': 3450, 'happy;neutral;sad': 3460, 'neutral;sad;surprise': 4560, 'angry;disgust;neutral': 7140, 'angry;fear;neutral': 7240, 'angry;fear;surprise': 7250, 'angry;happy;neutral': 7340, 'angry;neutral;surprise': 7450, 'angry;disgust;fear;neutral;sad': 10000, 'angry;disgust;fear;neutral;surprise': 2

In [8]:
# encoding Emotion for whole data
# 사전에 실제로 encoding한 끝 수가 마지막 linear layer의 끝자리랑 맞아야 합니다. 아니면 CUDA error: CUBLAS_STATUS_EXECUTION_FAILED가 나는 것 같아요.
# 예를 들어, label이 0~9, 11,13이렇게 12개가 되었어도, 0~13은 14개니까 마지막 레이어에서 14개 unit을 받아야 multiclass classification이 에러없이 진행됩니다!
# 데이터에서 정답 라벨 인코딩: ['neutral', 'happy', 'surprise', 'disgust', 'angry', 'sad', 'fear']
# 이 순서를 지켜서 라벨링을 해야함
decode_dict = {b:i for i, b in encode_dict.items()}
print(encode_dict, '\n', decode_dict)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6, 'angry;disgust': 10, 'angry;fear': 20, 'angry;neutral': 40, 'angry;surprise': 50, 'angry;sad': 60, 'disgust;fear': 120, 'disgust;happy': 130, 'disgust;neutral': 140, 'disgust;surprise': 150, 'disgust;sad': 160, 'fear;happy': 230, 'fear;neutral': 240, 'fear;surprise': 250, 'fear;sad': 260, 'happy;neutral': 340, 'happy;surprise': 350, 'happy;sad': 360, 'neutral;surprise': 450, 'neutral;sad': 460, 'sad;surprise': 560, 'angry;happy': 730, 'disgust;neutral;surprise': 1450, 'disgust;neutral;sad': 1460, 'fear;happy;neutral': 2340, 'fear;neutral;surprise': 2450, 'disgust;happy;neutral': 3410, 'happy;neutral;surprise': 3450, 'happy;neutral;sad': 3460, 'neutral;sad;surprise': 4560, 'angry;disgust;neutral': 7140, 'angry;fear;neutral': 7240, 'angry;fear;surprise': 7250, 'angry;happy;neutral': 7340, 'angry;neutral;surprise': 7450, 'angry;disgust;fear;neutral;sad': 10000, 'angry;disgust;fear;neutral;surprise': 2

## Annotation Encoding

In [9]:
kemdy19_annot.Emotion = list(kemdy19_annot.Emotion.map(encode_dict))
kemdy20_annot.Emotion = list(kemdy20_annot.Emotion.map(encode_dict))
kemdy19_annot.Emotion[:3], kemdy20_annot.Emotion[:3]


(0    6
 1    2
 2    0
 Name: Emotion, dtype: int64,
 0    4
 1    4
 2    4
 Name: Emotion, dtype: int64)

# EDA, TEMP Padding

In [10]:
import math
def add_padding(pd_series, length = 50):
    if isinstance(pd_series, float):
        if math.isnan(pd_series):
            return np.zeros(10)
    if len(pd_series) < length:
        pd_series = np.concatenate([pd_series, np.zeros(length - len(pd_series))])
        return np.array(pd_series)
    elif len(pd_series) == length:
        return np.array(pd_series)
    elif len(pd_series) > length:
        pd_series = pd_series[:length]
        return np.array(pd_series)

In [11]:
kemdy19_annot.EDA.map(len).describe()

count    9008.000000
mean       22.596470
std        19.693268
min         1.000000
25%         9.000000
50%        17.000000
75%        30.000000
max       314.000000
Name: EDA, dtype: float64

In [12]:
kemdy19_annot['Scaled EDA'] = kemdy19_annot['Scaled EDA'].apply(add_padding)
kemdy20_annot['Scaled EDA'] = kemdy20_annot['Scaled EDA'].apply(add_padding)
kemdy19_annot['Scaled TEMP'] = kemdy19_annot['Scaled TEMP'].apply(add_padding)
kemdy20_annot['Scaled TEMP'] = kemdy20_annot['Scaled TEMP'].apply(add_padding)
# check
kemdy20_annot['Scaled EDA'][10], kemdy19_annot['Scaled EDA'][10], kemdy20_annot['Scaled TEMP'][3],kemdy19_annot['Scaled TEMP'][15]

(array([ 0.33301774,  0.55311825, -0.16032486, -0.10719511,  0.09773054,
         0.10531797,  0.12809209,  0.40890977,  0.23435159, -0.29694591,
        -0.36525051, -0.25140359, -0.46391666, -0.14514408,  0.05978157,
         0.74286905,  0.61383542, -3.12036271,  2.16975526,  1.91929321,
        -0.29693999, -0.19827383, -0.94967184,  0.18880927,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]),
 array([-0.21123698, -0.18926761, -0.18665215, -0.1861293 , -0.1861293 ,
        -0.18874476, -0.20757542, -0.25517533, -0.29074444, -0.29179054,
        -0.30434459, -0.24105233, -0.24628284, -0.48480603, -0.52822112,
        -0.60860162, -0.77337071, -0.85676544, -0

In [13]:
lengt_wav = 0
lengt_txt = 0
for i,j in zip(kemdy19_emb[0], kemdy19_emb[1]):
    lengt_wav += len(kemdy19_emb[0][i])
    lengt_txt += len(kemdy19_emb[1][j])
    # lengt += len(i)
print(f'session 19 length: wav - {lengt_wav}, txt - {lengt_txt}')

lengt_wav = 0
lengt_txt = 0
for i,j in zip(kemdy20_emb[0], kemdy20_emb[1]):
    lengt_wav += len(kemdy20_emb[0][i])
    lengt_txt += len(kemdy20_emb[1][j])
    
print(f'session 20 length: wav - {lengt_wav}, txt - {lengt_txt}')

session 19 length: wav - 9008, txt - 9008
session 20 length: wav - 12715, txt - 12715


## Session pick 
- test(.2), validation(.2), train(.8)

In [14]:
def choice_and_remove_list(original_list, k = 8):
    removed_new_list = []
    chosen_list = random.sample(original_list, k = k)
    for session in original_list:
        if session in chosen_list:
            pass
        else:
            removed_new_list.append(session) 
    return sorted(removed_new_list), sorted(chosen_list)

In [15]:
# session을 train vs test&val로 나눠줌
session_20_lst = ['Sess0' + str(i+1) if i < 9 else 'Sess' + str(i+1) for i in range(40)]
session_20_lst.remove('Sess12')
session_20_lst.remove('Sess17')
sessions_20_train_lst, sessions_20_test_lst = choice_and_remove_list(session_20_lst, k = 8)
sessions_20_train_lst, sessions_20_val_lst = choice_and_remove_list(sessions_20_train_lst, k = 8)
print(sessions_20_train_lst, sessions_20_test_lst, sessions_20_val_lst, sep = '\n')

session_19_lst = ['Sess0' + str(i+1) if i < 9 else 'Sess' + str(i+1) for i in range(20)]
sessions_19_train_lst, sessions_19_test_lst = choice_and_remove_list(session_19_lst, k = 4)
sessions_19_train_lst, sessions_19_val_lst = choice_and_remove_list(sessions_19_train_lst, k = 4)
print('\n', sessions_19_train_lst,sessions_19_test_lst, sessions_19_val_lst, sep = '\n')

['Sess02', 'Sess03', 'Sess04', 'Sess06', 'Sess07', 'Sess09', 'Sess10', 'Sess11', 'Sess13', 'Sess18', 'Sess20', 'Sess23', 'Sess25', 'Sess26', 'Sess28', 'Sess29', 'Sess30', 'Sess33', 'Sess34', 'Sess35', 'Sess36', 'Sess38']
['Sess08', 'Sess16', 'Sess19', 'Sess21', 'Sess22', 'Sess24', 'Sess37', 'Sess39']
['Sess01', 'Sess05', 'Sess14', 'Sess15', 'Sess27', 'Sess31', 'Sess32', 'Sess40']


['Sess01', 'Sess02', 'Sess03', 'Sess04', 'Sess05', 'Sess06', 'Sess07', 'Sess09', 'Sess10', 'Sess11', 'Sess13', 'Sess16']
['Sess14', 'Sess15', 'Sess17', 'Sess19']
['Sess08', 'Sess12', 'Sess18', 'Sess20']


## data reconstruct
- dict - session - txt, wav, segment_id

In [16]:
kemdy20_emb_new = {}
for session in kemdy20_emb[0].keys():
    kemdy20_emb_new[session] = {}
    kemdy20_emb_new[session]['wav'] = kemdy20_emb[0][session]
    kemdy20_emb_new[session]['txt'] = kemdy20_emb[1][session]
    kemdy20_emb_new[session]['segment_id'] = kemdy20_annot['Segment ID'][kemdy20_annot['Segment ID'].str.startswith(session)]
    # print(session, len(kemdy20_emb_new[session]['wav']), len(kemdy20_emb_new[session]['txt']), len(kemdy20_emb_new[session]['segment_id']))

# print('\n')
kemdy19_emb_new = {}
for session in kemdy19_emb[0].keys():
    kemdy19_emb_new[session] = {}
    kemdy19_emb_new[session]['wav'] = kemdy19_emb[0][session]
    kemdy19_emb_new[session]['txt'] = kemdy19_emb[1][session]
    kemdy19_emb_new[session]['segment_id'] = kemdy19_annot['Segment ID'][kemdy19_annot['Segment ID'].str.startswith(session)]
    # print(session, len(kemdy19_emb_new[session]['wav']), len(kemdy19_emb_new[session]['txt']), len(kemdy19_emb_new[session]['segment_id']))


In [17]:
# 나눠준 세션을 emb(wav, text순), annot set에 적용
def get_data_by_session(data, session_lst):
    if isinstance(data, pd.DataFrame):
        print('dataframe')
        for idx, session in enumerate(session_lst):
            if idx == 0:
                dataframe = data[data['Segment ID'].str.startswith(session)]
            else:
                dataframe = pd.concat([dataframe, data[data['Segment ID'].str.startswith(session)]])
        return dataframe
    
    elif isinstance(data, dict):
        print('dict')
        emb_data = {}
        emb_data['wav'] = []
        emb_data['txt'] = []
        emb_data['segment_id'] = []
        for session in session_lst:
            emb_data['wav'].extend(data[session]['wav'])
            emb_data['txt'].extend(data[session]['txt'])
            emb_data['segment_id'].extend(data[session]['segment_id'])            
        return emb_data

In [18]:
kemdy19_annot_train = get_data_by_session(kemdy19_annot, sessions_19_train_lst)
kemdy19_annot_test = get_data_by_session(kemdy19_annot, sessions_19_test_lst)
kemdy19_annot_val = get_data_by_session(kemdy19_annot, sessions_19_val_lst)

kemdy20_annot_train = get_data_by_session(kemdy20_annot, sessions_20_train_lst)
kemdy20_annot_test = get_data_by_session(kemdy20_annot, sessions_20_test_lst)
kemdy20_annot_val = get_data_by_session(kemdy20_annot, sessions_20_val_lst)

dataframe
dataframe
dataframe
dataframe
dataframe
dataframe


In [19]:
# session 정보가 없어집니다. 
kemdy19_emb_train = get_data_by_session(kemdy19_emb_new, sessions_19_train_lst)
kemdy19_emb_test = get_data_by_session(kemdy19_emb_new, sessions_19_test_lst)
kemdy19_emb_val = get_data_by_session(kemdy19_emb_new, sessions_19_val_lst)

kemdy20_emb_train = get_data_by_session(kemdy20_emb_new, sessions_20_train_lst)
kemdy20_emb_test = get_data_by_session(kemdy20_emb_new, sessions_20_test_lst)
kemdy20_emb_val = get_data_by_session(kemdy20_emb_new, sessions_20_val_lst)

dict
dict
dict
dict
dict
dict


In [20]:
# kemdy 19 - 9008이 정상
print(len(kemdy19_emb_train['wav']), len(kemdy19_emb_train['txt']), len(kemdy19_emb_train['segment_id']))
print(len(kemdy19_emb_test['wav']), len(kemdy19_emb_test['txt']), len(kemdy19_emb_test['segment_id']))
print(len(kemdy19_emb_val['wav']), len(kemdy19_emb_val['txt']), len(kemdy19_emb_val['segment_id']))
print(len(kemdy19_emb_train['wav']) + len(kemdy19_emb_test['wav']) + len(kemdy19_emb_val['wav']))

5138 5138 5138
1897 1897 1897
1973 1973 1973
9008


In [21]:
# kemdy 20 - 12715가 정상
print(len(kemdy20_emb_train['wav']), len(kemdy20_emb_train['txt']), len(kemdy20_emb_train['segment_id']))
print(len(kemdy20_emb_test['wav']), len(kemdy20_emb_test['txt']), len(kemdy20_emb_test['segment_id']))
print(len(kemdy20_emb_val['wav']), len(kemdy20_emb_val['txt']), len(kemdy20_emb_val['segment_id']))
print(len(kemdy20_emb_train['wav']) + len(kemdy20_emb_test['wav'])+ len(kemdy20_emb_val['wav']))

7080 7080 7080
2740 2740 2740
2895 2895 2895
12715


In [22]:
print(kemdy19_emb_train.keys(), kemdy19_emb_test.keys(), kemdy19_emb_val.keys(),
      kemdy20_emb_train.keys(),kemdy20_emb_test.keys(), kemdy20_emb_val.keys(),sep = '\n')

dict_keys(['wav', 'txt', 'segment_id'])
dict_keys(['wav', 'txt', 'segment_id'])
dict_keys(['wav', 'txt', 'segment_id'])
dict_keys(['wav', 'txt', 'segment_id'])
dict_keys(['wav', 'txt', 'segment_id'])
dict_keys(['wav', 'txt', 'segment_id'])


## Train Data neutral pick하기

In [23]:
# 각 데이터 셋에서 몇개 뽑아야 되는지 계산 neutral: 4
target_neutral_num = 5000

target_neutral_num_19 = int(target_neutral_num / (Counter(kemdy19_annot['Emotion'])[4] + Counter(kemdy20_annot['Emotion'])[4]) * Counter(kemdy19_annot['Emotion'])[4])
target_neutral_num_20 = target_neutral_num - target_neutral_num_19
target_neutral_num_19, target_neutral_num_20

(1241, 3759)

In [24]:
# 각 train dataset에서 뽑아야 되는 갯수만큼 랜덤으로 뽑아옴
kemdy19_annot_train_not_neut = kemdy19_annot_train[kemdy19_annot_train['Emotion'] != 4]
kemdy20_annot_train_not_neut = kemdy20_annot_train[kemdy20_annot_train['Emotion'] != 4]

kemdy19_annot_train_neut = kemdy19_annot_train[kemdy19_annot_train['Emotion'] == 4].sample(target_neutral_num_19)
kemdy20_annot_train_neut = kemdy20_annot_train[kemdy20_annot_train['Emotion'] == 4].sample(target_neutral_num_20)
len(kemdy19_annot_train_not_neut), len(kemdy19_annot_train_neut), len(kemdy20_annot_train_not_neut), len(kemdy20_annot_train_neut)

(3115, 1241, 1196, 3759)

## embedding, Test and validation dataset 합치기

In [25]:
emb_test_final = {}
emb_test_final['wav'] = []
emb_test_final['txt'] = []
emb_test_final['segment_id'] = []

emb_test_final['wav'] = kemdy19_emb_test['wav']
emb_test_final['wav'].extend(kemdy20_emb_test['wav'])
print(len(kemdy19_emb_test['wav']), len(emb_test_final['wav']))

emb_test_final['txt'] = kemdy19_emb_test['txt']
emb_test_final['txt'].extend(kemdy20_emb_test['txt'])
print(len(kemdy19_emb_test['txt']), len(emb_test_final['txt']))

emb_test_final['segment_id'] = kemdy19_emb_test['segment_id']
emb_test_final['segment_id'].extend(kemdy20_emb_test['segment_id'])
print(len(kemdy19_emb_test['segment_id']), len(emb_test_final['segment_id']))

print(len(emb_test_final['wav']), len(emb_test_final['txt']), len(emb_test_final['segment_id']))

4637 4637
4637 4637
4637 4637
4637 4637 4637


In [26]:
emb_val_final = {}
emb_val_final['wav'] = []
emb_val_final['txt'] = []
emb_val_final['segment_id'] = []

emb_val_final['wav'] = kemdy19_emb_val['wav']
emb_val_final['wav'].extend(kemdy20_emb_val['wav'])

emb_val_final['txt'] = kemdy19_emb_val['txt']
emb_val_final['txt'].extend(kemdy20_emb_val['txt'])

emb_val_final['segment_id'] = kemdy19_emb_val['segment_id']
emb_val_final['segment_id'].extend(kemdy20_emb_val['segment_id'])

print(len(emb_val_final['wav']), len(emb_val_final['txt']), len(emb_val_final['segment_id']))

4868 4868 4868


In [27]:
annot_train_final = pd.concat([kemdy19_annot_train_neut, kemdy20_annot_train_neut, kemdy19_annot_train_not_neut, kemdy20_annot_train_not_neut])
annot_test_final = pd.concat([kemdy19_annot_test, kemdy20_annot_test])
annot_val_final = pd.concat([kemdy19_annot_val, kemdy20_annot_val])

annot_train_final.reset_index(drop=True, inplace=True)
annot_test_final.reset_index(drop=True, inplace=True)
annot_val_final.reset_index(drop=True, inplace=True)

print(len(annot_train_final), len(annot_test_final), len(annot_val_final))

9311 4637 4868


In [28]:
# train dataset neutral 4000개로 랜덤 뽑은 것 생성
emb_train_final = {}
emb_train_final['wav'] = []
emb_train_final['txt'] = []
emb_train_final['segment_id'] = []
for segment_annot_id in kemdy19_annot_train_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy19_emb_train['wav'], kemdy19_emb_train['txt'], kemdy19_emb_train['segment_id']):
        if segment_annot_id == segment_emb_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
            
for segment_annot_id in kemdy19_annot_train_not_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy19_emb_train['wav'], kemdy19_emb_train['txt'], kemdy19_emb_train['segment_id']):
        if segment_annot_id == segment_emb_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
        

for segment_annot_id in kemdy20_annot_train_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy20_emb_train['wav'], kemdy20_emb_train['txt'], kemdy20_emb_train['segment_id']):
        if segment_emb_id == segment_annot_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
        
for segment_annot_id in kemdy20_annot_train_not_neut['Segment ID']:
    for wav, txt, segment_emb_id in zip(kemdy20_emb_train['wav'], kemdy20_emb_train['txt'], kemdy20_emb_train['segment_id']):
        if segment_annot_id == segment_emb_id:
            emb_train_final['wav'].append(wav)
            emb_train_final['txt'].append(txt)
            emb_train_final['segment_id'].append(segment_emb_id)
            
print(len(emb_train_final['wav']), len(emb_train_final['txt']), len(emb_train_final['segment_id']))

9311 9311 9311


# torch dataset 만들기
- 참고: https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

In [29]:
torch.set_default_dtype(torch.float32)
print(torch.get_default_dtype())

torch.float32


In [30]:
class EtriDataset(Dataset):
    def __init__(self, file_names, 
                 text_embeddings, 
                 wav_embeddings, 
                 Temp,
                 EDA,
                 Emotion,
                 Emotion_vec, 
                 Arousal, 
                 Valence):
        self.file_names = file_names
        self.text_embeddings = text_embeddings
        self.wav_embeddings = wav_embeddings
        self.temp = Temp
        self.eda = EDA
        self.label_emotion = Emotion
        self.label_emotion_vec = Emotion_vec
        self.label_arousal = Arousal
        self.label_valence = Valence
        
        
    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        text_embeddings = self.text_embeddings[idx]
        wav_embeddings = self.wav_embeddings[idx]
        temp = self.temp[idx]
        eda = self.eda[idx]
        label_emotion = self.label_emotion[idx]
        label_emotion_ext = self.label_emotion_vec[idx]
        label_arousal = self.label_arousal[idx]
        label_valence = self.label_valence[idx]
        return text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence

In [31]:
annot_train_final.columns

Index(['Segment ID', 'Emotion', 'Valence', 'Arousal', 'emotion_vector',
       'valence_vector', 'arousal_vector', 'EDA', 'TEMP', 'EDA length',
       'TEMP length', 'Scaled EDA', 'Scaled TEMP'],
      dtype='object')

In [32]:
annot_train_final['Scaled EDA']

0       [-1.9738758426167797, -1.9116486245641493, -1....
1       [-0.4630346865653657, -0.10242922491996066, -0...
2       [-0.6250026128208589, -0.6098925796417516, -0....
3       [1.1610535088678586, 1.1465380430005419, 1.161...
4       [0.17325996510114683, -0.2880033940502193, -0....
                              ...                        
9306    [1.0248824293032237, -0.8716879365334415, -0.1...
9307    [0.9738374515615607, -0.011967660642870578, -0...
9308    [-0.011967660642870578, -0.011967660642870578,...
9309    [-0.011967660642870578, -0.011967660642870578,...
9310    [-0.16042779153088987, -0.16061282278609879, 0...
Name: Scaled EDA, Length: 9311, dtype: object

In [33]:
# data load 및 나누기: https://076923.github.io/posts/Python-pytorch-11/

# annot_train_final, annot_test_final, annot_val_final
# emb_train_final, emb_test_final, emb_val_final

# session을 통합시킨 데이터 셋을 만들었을 때
dataset_train = EtriDataset(file_names = annot_train_final['Segment ID'],
                      text_embeddings = torch.stack(emb_train_final['txt']),
                      wav_embeddings = torch.stack(emb_train_final['wav']),
                      Emotion = annot_train_final['Emotion'],
                      Arousal = annot_train_final['Arousal'],
                      Valence = annot_train_final['Valence'],
                      EDA = torch.Tensor(annot_train_final['Scaled EDA']), 
                      Temp = torch.Tensor(annot_train_final['Scaled TEMP']), 
                      Emotion_vec = torch.Tensor(annot_train_final['emotion_vector'])) 


dataset_test = EtriDataset(file_names = annot_test_final['Segment ID'],
                      text_embeddings = torch.stack(emb_test_final['txt']),
                      wav_embeddings = torch.stack(emb_test_final['wav']),
                      Emotion = annot_test_final['Emotion'],
                      Arousal = annot_test_final['Arousal'],
                      Valence = annot_test_final['Valence'],
                      EDA = torch.Tensor(annot_test_final['Scaled EDA']), 
                      Temp = torch.Tensor(annot_test_final['Scaled TEMP']), 
                      Emotion_vec = torch.Tensor(annot_test_final['emotion_vector']))

dataset_val = EtriDataset(file_names = annot_val_final['Segment ID'],
                      text_embeddings = torch.stack(emb_val_final['txt']),
                      wav_embeddings = torch.stack(emb_val_final['wav']),
                      Emotion = annot_val_final['Emotion'],
                      Arousal = annot_val_final['Arousal'],
                      Valence = annot_val_final['Valence'],
                      EDA = torch.Tensor(annot_val_final['Scaled EDA']), 
                      Temp = torch.Tensor(annot_val_final['Scaled TEMP']), 
                      Emotion_vec = torch.Tensor(annot_val_final['emotion_vector'])) 


/tmp/ipykernel_362029/245938389.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  EDA = torch.Tensor(annot_train_final['Scaled EDA']),


In [34]:
print(f"Training Data Size : {len(dataset_train)}")
print(f"Validation Data Size : {len(dataset_val)}")
print(f"Testing Data Size : {len(dataset_test)}")

Training Data Size : 9311
Validation Data Size : 4868
Testing Data Size : 4637


In [35]:
train_dataloader = DataLoader(dataset_train, batch_size=512, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(dataset_val, batch_size=128, shuffle=True, drop_last=True)
test_dataloader = DataLoader(dataset_test, batch_size=128, shuffle=True, drop_last=True)

# NetWork 만들기

In [36]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [37]:
class MLPNetwork_pre(nn.Module):
    def __init__(self, input_length, input_width):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_length*input_width, 768)
        self.gelu1 = nn.GELU()
        self.bn1 = nn.BatchNorm1d(768)
        self.fc2 = nn.Linear(768, 512)
        self.gelu2 = nn.GELU()
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 32)
        self.gelu3 = nn.GELU()
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.gelu1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.gelu2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        output = self.gelu3(x)
        return output
    
class ConvNetwork_pre(nn.Module):
    def __init__(self, input_channel):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels = input_channel, out_channels= 32, kernel_size = 3, padding = 1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 20, kernel_size = 3, padding = 1)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        output = self.relu2(x)
        return output

class ConvNetwork_final(nn.Module):
    def __init__(self, input_channel):
        super().__init__()
        self.conv2d_1 = nn.Conv2d(in_channels = input_channel, out_channels = 64, kernel_size=2)
        self.leakyrelu_1 = nn.LeakyReLU()
        self.maxpool2d_1 = nn.MaxPool2d(2)
        self.conv2d_2 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size=2)
        self.leakyrelu_2 = nn.LeakyReLU()
        self.maxpool2d_2 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(896, 64)
        self.leakyrelu_3 = nn.LeakyReLU()
        self.batchnorm = nn.BatchNorm1d(64)
        self.drop = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(64, 7)
        
    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.leakyrelu_1(x)
        x = self.maxpool2d_1(x)
        x = self.conv2d_2(x)
        x = self.leakyrelu_2(x)
        x = self.maxpool2d_2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.leakyrelu_3(x)
        x = self.batchnorm(x)
        x = self.drop(x)
        output = self.fc2(x)  
        return output
        



In [38]:
class TensorFusionMixer(nn.Module):
    def __init__(self, ModelA, ModelB, ModelC, ModelD, ModelE):
        super().__init__()
        self.ModelA = ModelA
        self.ModelB = ModelB
        self.ModelC = ModelC
        self.ModelD = ModelD
        self.Model_cnn_final = ModelE
        self.softmax = nn.Softmax(dim = 1)
        # self.softmax = nn.Softmax(dim=1)
        
    def tensor_fusion(self, batch_arr1, batch_arr2, batch_arr3):
        fusion_matrix_lst = []
        for i, (arr1, arr2, arr3) in enumerate(zip(batch_arr1, batch_arr2, batch_arr3)):
            arr1 = arr1.unsqueeze(-1).unsqueeze(-1)
            arr2 = arr2.unsqueeze(0).unsqueeze(-1)
            arr3 = arr3.squeeze().unsqueeze(0).unsqueeze(0)
            
            # outer_matrix = torch.einsum('i,j,kp->ijk', arr1, arr2, arr3)
            outer_matrix = torch.kron(torch.kron(arr1,arr2), arr3)
            l, w, d = outer_matrix.shape
            
            outer_matrix = outer_matrix.view(-1, l, w, d)
            fusion_matrix_lst.append(outer_matrix)
            
        fusion_matrix = torch.concat(fusion_matrix_lst)
        
        return fusion_matrix
        
    def forward(self, x1, x2, x3, x4):
        x1 = self.ModelA(x1)
        x2 = self.ModelB(x2)
        x3 = self.ModelC(x3)
        x4 = self.ModelD(x4)
        x5 = torch.cat([x3,x4], dim=0)
        
        fusion_matrix = self.tensor_fusion(x1, x2, x5) 
        x = self.Model_cnn_final(fusion_matrix) # 새로운 emotion사용
        output = self.softmax(x)
        return output

In [39]:
annot_train_final['emotion_vector'][0:10]

0     [0, 0, 0, 1, 9, 0, 0]
1     [0, 0, 0, 0, 9, 1, 0]
2     [0, 0, 0, 1, 9, 0, 0]
3    [0, 0, 0, 0, 10, 0, 0]
4     [0, 0, 0, 0, 8, 2, 0]
5     [0, 0, 0, 0, 9, 1, 0]
6     [0, 4, 0, 0, 5, 0, 1]
7     [0, 0, 2, 0, 4, 2, 2]
8     [0, 0, 0, 0, 8, 0, 2]
9     [0, 0, 0, 2, 8, 0, 0]
Name: emotion_vector, dtype: object

In [40]:
# txt_input_length, txt_input_width = raw_dataset[session]['text_embeddings'][0].shape | 마지막엔 지울 것
# _, wav_input_length, wav_input_width = raw_dataset[session]['wav_embeddings'][0].shape
txt_input_length, txt_input_width = torch.Tensor(emb_train_final['txt'][0]).shape
wav_input_length, wav_input_width = torch.Tensor(emb_train_final['wav'][0]).shape
temp_input_length = annot_train_final['Scaled EDA'][0].shape[0]
eda_input_length = annot_train_final['Scaled TEMP'][0].shape[0]

print(txt_input_length, txt_input_width, wav_input_length, wav_input_width)

80 768 149 1024


In [41]:

# tf_mixer에 들어갈 wav mlp, txt mlp 선언
model_mlp_txt = MLPNetwork_pre(txt_input_length,txt_input_width).to(device)
model_mlp_wav = MLPNetwork_pre(wav_input_length,wav_input_width).to(device)
model_conv_temp = ConvNetwork_pre(temp_input_length).to(device)
model_conv_eda = ConvNetwork_pre(eda_input_length).to(device)

model_cnn_final = ConvNetwork_final(32).to(device)

# 최종 모델 선언
model_tf_cnn_mixer = TensorFusionMixer(ModelA = model_mlp_txt, 
                                   ModelB = model_mlp_wav,
                                   ModelC = model_conv_temp,
                                   ModelD = model_conv_eda,
                                   ModelE = model_cnn_final).to(device)

# model 병렬 학습 처리
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_mlp_txt = nn.DataParallel(model_mlp_txt).to(device)
    model_mlp_wav = nn.DataParallel(model_mlp_wav).to(device)
    model_conv_temp = nn.DataParallel(model_conv_temp).to(device)
    model_conv_eda = nn.DataParallel(model_conv_eda).to(device)
    model_tf_cnn_mixer = nn.DataParallel(model_tf_cnn_mixer).to(device)
print(model_tf_cnn_mixer)

Let's use 4 GPUs!
DataParallel(
  (module): TensorFusionMixer(
    (ModelA): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=61440, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=True)
      (gelu2): GELU(approximate='none')
      (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc3): Linear(in_features=512, out_features=32, bias=True)
      (gelu3): GELU(approximate='none')
    )
    (ModelB): MLPNetwork_pre(
      (flatten): Flatten(start_dim=1, end_dim=-1)
      (fc1): Linear(in_features=152576, out_features=768, bias=True)
      (gelu1): GELU(approximate='none')
      (bn1): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (fc2): Linear(in_features=768, out_features=512, bias=T

# 학습을 위한 train, test method 만들기

In [42]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)    
    # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
    for batch, (X_txt, X_wav, X_temp, X_eda, 
                    label_emotion, label_emotion_vec, label_arousal, label_valence) in enumerate(dataloader): 
        y = label_emotion_vec # 라벨을 변경하고자 하면 이 변수만 바꿔주면 나머지는 y로 적용
        # 예측 오류 계산 
        X_txt, X_wav, X_temp, X_eda, y= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),y.type(torch.float16).to(device)
        
        X_temp = X_temp.unsqueeze(dim=-1)
        X_eda = X_eda.unsqueeze(dim=-1)
        
        pred = model(X_txt, X_wav, X_temp, X_eda)
        y = F.softmax(y, dim = 1)
        
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.mean().backward() # weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.mean().item(), batch * len(X_txt) # weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

In [43]:
def test(dataloader, model, loss_fn, mode = 'test'):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    f1 = F1Score(task= 'multiclass', num_classes=7).to(device)   # 바로 multiclassification할 경우
    preds = []
    targets = []
    
    with torch.no_grad():
        # data 순서: text_embeddings, wav_embeddings, temp, eda, label_emotion, label_emotion_ext, label_arousal, label_valence
        for batch, (X_txt, X_wav, X_temp, X_eda, 
                        label_emotion, label_emotion_vec, label_arousal, label_valence) in enumerate(dataloader): 
            y = label_emotion_vec # 라벨을 변경하고자 하면 이 변수만 바꿔주면 나머지는 y로 적용
            # 예측 오류 계산
            X_txt, X_wav, X_temp, X_eda, y= X_txt.to(device), X_wav.to(device), X_temp.to(device), X_eda.to(device),y.type(torch.float16).to(device)
            
            X_temp = X_temp.unsqueeze(dim=-1)
            X_eda = X_eda.unsqueeze(dim=-1)
            
            pred = model(X_txt, X_wav, X_temp, X_eda)
            y = F.softmax(y, dim = 1)
            # pred.topk(2)
            preds.append(pred.argmax(1)) # multi regression후 classification으로 변환할 경우
            targets.append(label_emotion) # classification을 할 경우 언제나 사용
            # print('예측라벨분포:',pred[:2], '정답라벨 분포:', label_emotion_ext[:2], '예측정답:', pred.argmax(1)[:2],'정답:', label_emotion[:2])
            # print('예측:', pred.argmax(1).tolist()[:2],'\n', '정답:', label_emotion.tolist()[:2])
            # https://discuss.pytorch.org/t/loss-backward-raises-error-grad-can-be-implicitly-created-only-for-scalar-outputs/12152/6
            test_loss += loss_fn(pred, y).mean().item()# weighted MSE를 사용할 경우 중간에 sum() or mean()을 넣어줌 
            
            correct += (pred.argmax(1) == label_emotion.to(device)).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    if mode == 'test':
        print(torch.cat(preds), torch.cat(preds).shape)
        print("f1 score: ", f1(torch.cat(preds).to(device), torch.cat(targets).to(device)))
        print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    elif mode == 'val':
        print(f"Validation Error: Accuracy: {(100*correct):>0.1f}%, Avg val loss: {test_loss:>8f} \n")

# 학습시키기

In [44]:
# weigted loss for imbalance data: https://naadispeaks.wordpress.com/2021/07/31/handling-imbalanced-classes-with-weighted-loss-in-pytorch/
# weight 계산
single_emotion = [0,1,2,3,4,5,6]
total_obs = 0
for i in single_emotion:
    total_obs += Counter(annot_train_final['Emotion'])[i]
print('total obs: ', total_obs)

weight_for_class = []
print(Counter(annot_train_final['Emotion']))
for key, value in sorted(Counter(annot_train_final['Emotion']).items()):
    if key in single_emotion:
        print(f'{key} is in single emotion')
        if key == 'neutral':
            weight_for_class.append(1 - (1440/total_obs))
        else:
            weight_for_class.append(1 - (value/total_obs))
weight_for_class = torch.Tensor(weight_for_class).type(torch.float16)
weight_for_class

total obs:  8671
Counter({4: 5000, 3: 1232, 0: 954, 6: 596, 5: 437, 2: 269, 340: 257, 1: 183, 40: 91, 450: 66, 460: 58, 140: 48, 10: 21, 350: 15, 250: 14, 240: 13, 50: 9, 20: 6, 60: 6, 560: 5, 230: 4, 260: 3, 7140: 3, 150: 2, 1450: 2, 3450: 2, 7340: 1, 7250: 1, 730: 1, 7240: 1, 7450: 1, 4560: 1, 2450: 1, 3460: 1, 130: 1, 20000: 1, 1460: 1, 10000: 1, 360: 1, 3410: 1, 2340: 1})
0 is in single emotion
1 is in single emotion
2 is in single emotion
3 is in single emotion
4 is in single emotion
5 is in single emotion
6 is in single emotion


tensor([0.8901, 0.9790, 0.9688, 0.8579, 0.4233, 0.9497, 0.9312],
       dtype=torch.float16)

In [45]:
class weighted_MSELoss(nn.Module):
    def __init__(self, weight):
        super().__init__()
        self.weight = weight.to(device)
    def forward(self,inputs,targets):
        return ((inputs - targets)**2) * self.weight

In [46]:
# # 지난 저장한 모델이 있다면
# PATH = './data/test_model.pkl'
# model_tf_mixer = torch.load(PATH)

In [47]:
# loss_fn = nn.CrossEntropyLoss(weight=weight_for_class).to(device)
# loss_fn = nn.CrossEntropyLoss().to(device) # weigth를 주기위해 위의 loss로 임시 변경
loss_fn = weighted_MSELoss(weight = weight_for_class).to(device) # multi target regression(감정별로 count 한 타겟)

In [50]:
lr = 1e-4

# optimizer = optim.SGD(model_tf_mixer.parameters(), lr=lr) # classification
optimizer = optim.Adam(model_tf_cnn_mixer.parameters(), lr=lr) # regression

## start training mlp fusion mixer

In [51]:
# Set the Training Parameters
epochs = 50
for epoch in range(epochs):
    print(f"---------------Epoch {epoch+1}----------------")
    train(train_dataloader, model_tf_cnn_mixer, loss_fn, optimizer)
    test(validation_dataloader, model_tf_cnn_mixer, loss_fn, mode = 'val')
print("Done!")

---------------Epoch 1----------------
loss: 0.055926  [    0/ 9311]
Validation Error: Accuracy: 25.1%, Avg val loss: 0.067268 

---------------Epoch 2----------------
loss: 0.067558  [    0/ 9311]
Validation Error: Accuracy: 52.6%, Avg val loss: 0.055508 

---------------Epoch 3----------------
loss: 0.047290  [    0/ 9311]
Validation Error: Accuracy: 58.6%, Avg val loss: 0.054345 

---------------Epoch 4----------------
loss: 0.040393  [    0/ 9311]
Validation Error: Accuracy: 48.9%, Avg val loss: 0.059643 

---------------Epoch 5----------------
loss: 0.039254  [    0/ 9311]
Validation Error: Accuracy: 42.9%, Avg val loss: 0.064502 

---------------Epoch 6----------------
loss: 0.029827  [    0/ 9311]
Validation Error: Accuracy: 48.7%, Avg val loss: 0.063351 

---------------Epoch 7----------------
loss: 0.026060  [    0/ 9311]
Validation Error: Accuracy: 39.8%, Avg val loss: 0.069778 

---------------Epoch 8----------------
loss: 0.027826  [    0/ 9311]
Validation Error: Accuracy: 

In [ ]:
# 실험을 위해 모델 저장
PATH = './data/model_multilabel_CNN.pkl'
torch.save(model_tf_cnn_mixer, PATH)

In [ ]:
test(test_dataloader, model_tf_cnn_mixer, loss_fn, mode = 'test')

AttributeError: 'list' object has no attribute 'to'

In [ ]:
for batch, (X_txt, X_wav, X_temp, X_eda, 
                        label_emotion, label_emotion_ext, label_arousal, label_valence) in list(enumerate(test_dataloader))[:4]:
    print(X_txt,X_wav,label_emotion)

tensor([[[-0.0372, -0.1859,  0.3214,  ..., -0.5023, -0.2907, -0.5705],
         [-0.3403, -0.6411,  0.2710,  ..., -0.4994,  0.1688, -0.6985],
         [-0.2038,  0.1151,  0.3303,  ...,  0.0532,  0.0731, -0.7255],
         ...,
         [ 0.0387,  0.1966,  0.1149,  ..., -0.5742, -0.1841, -0.6823],
         [ 0.3737,  0.0196, -0.0652,  ..., -0.3645,  0.1043, -0.9926],
         [ 0.4129, -0.1367,  0.0095,  ..., -0.4642,  0.2714, -0.9962]],

        [[ 1.3079,  0.3727,  1.6503,  ...,  0.9950, -0.2278,  1.1599],
         [ 1.6657, -0.2183,  2.2059,  ...,  0.7725,  1.6774,  0.4149],
         [-0.3042, -1.3660,  1.2436,  ...,  1.1886, -0.1148,  1.5758],
         ...,
         [ 0.9146, -0.8450,  1.8701,  ...,  0.0545, -0.1092,  1.0398],
         [ 0.9892, -0.2653,  1.9712,  ...,  0.7016,  0.0530,  0.9379],
         [ 0.9061, -0.3329,  1.8953,  ...,  0.7556,  0.1093,  0.9114]],

        [[-0.3704, -0.2984,  0.9155,  ..., -1.2681, -0.0433,  0.2413],
         [ 0.1247, -0.0254,  0.4202,  ..., -0

In [ ]:
probs = model_tf_mixer(X_txt.to(device), X_wav.to(device))
for i in torch.argmax(probs, dim=1):
    if decode_dict[int(i)] != 'neutral':
        print(decode_dict[int(i)])
    